In [2]:
from Modules import Preprocessor
from datetime import datetime, timedelta
import numpy as np
from urllib.request import urlopen 
from urllib.parse import urlencode, unquote, quote_plus
import urllib 
import json 
import os
import time
import pandas as pd

cwd = os.getcwd()
pp = Preprocessor(cwd)
pp.setFilename("국토안전관리원_건설안전사고사례_20221118.csv")
# pp.setFilename("국토안전관리원_20221118_after_temp.csv")
pp.openFile()
data = pp.data

def isNaN(num):
    return num != num


f = open("Data/key.txt", 'r')
api_url = f.readline()[:-2]
enKey = f.readline()
f.close()
enKey

api_url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"

In [13]:
list(pp.data)
# pp.data['시설물 소분류'].value_counts()
# pp.data

['사고명',
 '사고일시',
 '공공민간구분',
 '날씨',
 '온도',
 '습도',
 '시설물 대분류',
 '시설물 중분류',
 '시설물 소분류',
 '연면적',
 '지상층수',
 '지하층수',
 '공사종류',
 '인적사고종류(대분류)',
 '인적사고종류',
 '안전방호조치여부',
 '개인보호조치여부',
 '물적사고종류',
 '공종(대분류)',
 '공종(소분류)',
 '사고객체(대분류)',
 '사고객체(소분류)',
 '작업프로세스',
 '사고위치 장소',
 '사고위치 장소(직접입력)',
 '사고위치 부위',
 '사고위치 부위(직접입력)',
 '주원인 유형',
 '사고원인(주원인)',
 '사고원인(보조원인1)',
 '사고원인(보조원인2)',
 '구체적사고원인',
 '사망자',
 '내국인 사망자',
 '외국인 사망자',
 '남성 사망자',
 '여성 사망자',
 '10이상 20미만 사망자',
 '20이상 30미만 사망자',
 '30이상 40미만 사망자',
 '40이상 50미만 사망자',
 '50이상 60미만 사망자',
 '60이상 사망자',
 '부상자',
 '내국인 부상자',
 '외국인 부상자',
 '남성 부상자',
 '여성 부상자',
 '10이상 20미만 부상자',
 '20이상 30미만 부상자',
 '30이상 40미만 부상자',
 '40이상 50미만 부상자',
 '50이상 60미만 부상자',
 '60이상 부상자',
 '피해금액',
 '피해내용',
 '사고신고사유',
 '공사비',
 '해당공종 공사비',
 '공사시작일',
 '공사종료일',
 '해당공종 공사시작일',
 '해당공종 공사종료일',
 '낙찰율',
 '공정율',
 '작업자수',
 '안전관리계획',
 '설계안정성검토',
 '사고조사방법',
 '향후조치계획',
 '사고경위',
 '사고발생후 조치사항',
 '재발방지대책']

In [2]:

dateformat = "%Y%m%d"

# date = datetime.strptime('20191130', dateformat)
# date = date + timedelta(days=1)
# date.strftime(dateformat)

for idx, row in data.iterrows():
    if (row['온도'] == '미입력') or (float(row['온도']) > 40) or (float(row['온도']) < -15): 
        checkdata = (row['사고일시'].strip()).split(' ')[0].replace('-','')
        acdate = datetime.strptime(checkdata, dateformat)
        acdate.strftime(dateformat)
        # print(acdate.strftime(dateformat)," ", (acdate + timedelta(days=1)).strftime(dateformat))
        # print(type((acdate + timedelta(days=1)).strftime(dateformat)))
        params = f'?{quote_plus("ServiceKey")}={enKey}&' + urlencode({ 
            quote_plus("pageNo"): "1", 
            quote_plus("numOfRows"): "1", 
            quote_plus("dataType"): "JSON", 
            quote_plus("dataCd"): "ASOS",
            quote_plus("dateCd"): "DAY",
            quote_plus("startDt"): acdate.strftime(dateformat), 
            quote_plus("endDt"): (acdate + timedelta(days=1)).strftime(dateformat),
            quote_plus("stnIds"): "108"
            })
        # print(params)
        req = urllib.request.Request(api_url + params)
        response_body = urlopen(req).read()
        # time.sleep(1)
        # print(req)
        response_body = urlopen(req).read()
        response_data = json.loads(response_body) 
        weatherdata = response_data['response']['body']['items']['item'][0]
        avgtemp = weatherdata['avgTa']
        data.at[idx, '온도'] = float(avgtemp)
        print(avgtemp)
        
        
        
data['온도'].describe()

24.4
24.4
22.7
25.6
27.5
31.6
29.2
29.2
25.9
28.2
26.2
23.5
24.4
23.8
23.5
19.5
19.8
17.0
17.0
14.9
5.6
-0.1
4.6
14.8
10.6
17.1
16.5
14.3
19.1
17.4
23.0
24.4
24.9
24.9
22.3
23.8
25.6
25.0
23.8
23.8
24.4
23.7
24.3
27.1
27.1
27.5
27.1
27.1
24.3
26.4
23.3
23.4
21.1
21.3
21.3
21.3
19.7
19.1
20.4
20.4
17.2
17.2
16.9
16.2
15.1
15.2
13.5
12.9
13.6
4.9
4.9
13.5
13.5
13.5
10.4
6.5
4.6
4.6
5.6
4.1
-0.6
2.1
3.4
5.8
5.3
-8.3
-8.3
-2.1
-1.0
2.0
2.2
-4.2
-14.5
-12.2
-3.8
-6.3
-6.3
5.5
7.6
7.6
-2.6
-5.6
-3.2
-3.2
-3.2
1.4
1.2
-5.8
10.8
7.8
9.5
7.8
2.3
4.4
7.2
8.8
8.8
10.2
10.5
11.7
9.9
9.9
5.8
5.8
6.2
10.8
12.6
12.6
17.8
14.9
11.9
13.9
14.1
14.1
13.0
13.0
10.4
8.4
12.2
14.8
20.9
20.9
20.9
19.0
16.2
16.2
15.4
15.4
15.4
15.4
11.3
11.3
11.3
14.1
14.4
22.0
22.9
22.1
22.1
15.3
15.3
18.6
19.5
19.4
15.3
18.0
18.0
15.8
20.2
18.5
23.6
23.6
22.5
22.4
22.4
22.8
22.8
23.7
24.3
26.3
27.1
27.1
27.1
27.1
23.3
25.3
26.4
26.4
26.0
25.5
28.1
29.7
29.7
28.8
28.8
30.5
31.2
31.2
31.7
31.1
30.5
30.5
26.5
28.0
27.7
27.4
26

count     14289
unique      533
top          20
freq        820
Name: 온도, dtype: object

In [3]:
data.to_csv("Data/국토안전관리원_20221118_after_temp.csv")

In [8]:

for idx, row in data.iterrows():
    if row['시설물 대분류'] == '건축':
        if row['시설물 소분류'] == "공동주택" or row['시설물 소분류'] == "공장":
            data.at[idx, '시설물 대분류'] = row['시설물 소분류']
        else:
            data.at[idx, '시설물 대분류'] = '기타 건축'

# data = data.drop(['시설물 중분류', '시설물 소분류'], axis=1)
data= data.rename(columns={"시설물 대분류" : "시설물 분류"})
# data
data['시설물 분류'].value_counts()

기타 건축     5272
공동주택      4117
토목        2993
공장        1152
산업환경설비     619
조경         136
Name: 시설물 분류, dtype: int64

In [9]:
dateformat = "%Y-%m-%d %H:%M"
for idx, row in data.iterrows():
    checkdata = row['사고일시'].strip()
    data.at[idx,'Month'] = int(datetime.strptime(checkdata, dateformat).month)
    data.at[idx,'Time'] = int(datetime.strptime(checkdata, dateformat).hour)


print(data['Month'].value_counts())
data['Time'].value_counts()


7.0     1530
10.0    1365
4.0     1338
5.0     1221
3.0     1207
11.0    1202
6.0     1177
8.0     1176
12.0    1176
9.0     1133
2.0      922
1.0      842
Name: Month, dtype: int64


10.0    2036
14.0    1695
11.0    1684
15.0    1627
9.0     1594
8.0     1338
16.0    1310
13.0    1170
7.0      607
17.0     381
12.0     368
6.0      109
18.0     109
19.0      46
0.0       45
22.0      25
23.0      23
20.0      22
21.0      22
5.0       20
2.0       17
1.0       14
4.0       14
3.0       13
Name: Time, dtype: int64

In [20]:
for idx, row in data.iterrows():
    if row['인적사고종류(대분류)'] == '미입력' or row['인적사고종류(대분류)'] == '없음' or row['인적사고종류(대분류)'] == '분류불능':
        data.at[idx, '인적사고종류(대분류)'] = '기타'

# data = data.drop(['인적사고종류'], axis=1)
data= data.rename(columns={"인적사고종류(대분류)" : "인적사고종류"})
data['인적사고종류'].value_counts()
# data['인적사고종류']

넘어짐       3368
떨어짐       2801
물체에 맞음    2201
끼임        1558
기타        1518
절단, 베임    1016
부딪힘        949
깔림         302
질병         190
찔림         170
화상          95
교통사고        74
감전          34
질식          13
Name: 인적사고종류, dtype: int64

In [22]:
for idx, row in data.iterrows():
    if row['물적사고종류'] == '미입력' or row['물적사고종류'] == '없음' :
        data.at[idx, '물적사고종류'] = '기타'

for idx, row in data.iterrows():
    if row['인적사고종류'] == '기타' :
        data.at[idx, '사고종류'] = row['물적사고종류']
    else:
        data.at[idx, '사고종류'] = row['인적사고종류']

data['사고종류'].value_counts()

넘어짐       3368
떨어짐       2801
물체에 맞음    2201
끼임        1558
기타        1404
절단, 베임    1016
부딪힘        949
깔림         302
질병         190
찔림         170
화상          95
교통사고        74
전도          41
감전          34
화재          29
충돌          14
질식          13
붕괴          12
탈락           6
파열, 파단       6
낙하           4
폭발           2
Name: 사고종류, dtype: int64

In [23]:

for idx, row in data.iterrows():
    if row['10이상 20미만 사망자'] > 0:
        # print(data.at[idx, '20이상 30미만 사망자'])
        data.at[idx, '20이상 30미만 사망자'] += row['10이상 20미만 사망자']
        # print(data.at[idx, '20이상 30미만 사망자'])
    if row['10이상 20미만 부상자'] > 0:
        # print(data.at[idx, '20이상 30미만 부상자'])
        data.at[idx, '20이상 30미만 부상자'] += row['10이상 20미만 부상자']
        # print(data.at[idx, '20이상 30미만 부상자'])

# data = data.drop(['10이상 20미만 사망자', '10이상 20미만 부상자'], axis=1)
# data= data.rename(columns={"시설물 대분류" : "시설물 분류"})
# # data
# data['시설물 분류'].value_counts()

In [24]:
typechecklist = {
                '지붕 및 홈통공사' : '지붕 및 홈통공사',
                '철도 및 궤도공사': '철골공사',
                '교량공사': '철골공사',
                '철근콘크리트공사': '콘크리트공사',
                '프리캐스트 콘크리트공사': '콘크리트공사',
                '철골공사': '철골공사',
                '지정공사': '지정 및 기초공사',
                '말뚝공사': '지정 및 기초공사',
                '지반조사': '지정 및 기초공사',
                '지반개량공사': '지정 및 기초공사',
                '미장공사': '미장공사',
                '가설공사': '가설공사',
                '방수공사': '방수 및 방습공사',
                '건축물 부대공사': '내외벽공사',
                '해체 및 철거공사': '해체 및 재활용공사',
                '도장공사': '도장공사',
                '토공사': '토공사',
                '건축 토공사': '토공사',
                '타일 및 돌공사': '타일 및 테라코타공사',
                '조경공사': '타일 및 테라코타공사',
                '조적공사': '조적공사',
                '금속공사': '금속공사',
                '수장공사': '수장공사',
                '목공사': '목공사',
                '하천공사': '토공사',

                '기계설비공사' : '설비공사',
                '전기설비공사' : '설비공사',
                '산업설비공사' : '설비공사',
                '통신설비공사' : '설비공사',

                '관공사'  : '관공사',
                '관공사 부대공사' : '관공사',

                '도로 및 포장공사'  : '토목공사',
                '터널공사' : '토목공사',
                '항만공사'  : '토목공사',
                '강구조물공사'  : '토목공사',
                '댐 및 제방공사' : '토목공사',
            }                               
# 설비공사, 관공사, 토목공사는 공종별 위험도 데이터가 논문에 존재하지 않음
# 설비공사는 E/V공사 지표를 사용해도 될 것이라 예상
typelist = ['지붕 및 홈통공사','E/V공사','커튼월 공사','콘크리트공사','철골공사','지정 및 기초공사','내외벽공사','미장공사','가설공사','방수 및 방습공사','해체 및 재활용공사','도장공사','토공사','타일 및 테라코타공사','기타공사','조적공사','석공사','창호 및 유리공사','금속공사','수장공사','단열 및 방내화공사','목공사','기계설비공사']                                                      



for idx, row in data.iterrows():
    checkkeys = list(typechecklist.keys())

    if row['공종(소분류)'] in checkkeys:
        data.at[idx, '공종'] = typechecklist[row['공종(소분류)']]
    else:
        data.at[idx, '공종'] = '기타공사'

# data = data.drop(['인적사고종류'], axis=1)
# data= data.rename(columns={"인적사고종류(대분류)" : "인적사고종류"})
data['공종'].value_counts()

콘크리트공사         3959
기타공사           1804
가설공사           1586
설비공사           1092
토공사            1032
철골공사            707
해체 및 재활용공사      695
토목공사            567
관공사             481
수장공사            363
타일 및 테라코타공사     325
미장공사            315
내외벽공사           312
도장공사            196
목공사             173
조적공사            172
지정 및 기초공사       172
금속공사            129
방수 및 방습공사       110
지붕 및 홈통공사        99
Name: 공종, dtype: int64

In [25]:
costchecklist = {
                    '1,000만원 미만' : '공사규모_0-5',
                    '1,000만 ~ 2,000만원 미만' : '공사규모_0-5',
                    '2,000만 ~ 4,000만원 미만' : '공사규모_0-5',
                    '4,000만 ~ 1억원 미만' : '공사규모_0-5',
                    '1억 ~ 2억원 미만' : '공사규모_0-5',
                    '2억 ~ 3억원 미만' : '공사규모_0-5',
                    '3억 ~ 5억원 미만' : '공사규모_0-5',
                    '5억 ~ 10억원 미만' : '공사규모_5-10',
                    '10억 ~ 20억원 미만' : '공사규모_10-30',
                    '20억 ~ 50억원 미만' : '공사규모_30-50',
                    '50억 ~ 100억원 미만' : '공사규모_50-100',
                    '100억 ~ 150억원 미만' : '공사규모_100-500',
                    '150억 ~ 200억원 미만' : '공사규모_100-500',
                    '200억 ~ 300억원 미만' : '공사규모_100-500',
                    '300억 ~ 500억원 미만' : '공사규모_100-500',
                    '500억 ~ 1,000억원 미만' : '공사규모_500-1000',
                    '1,000억원 이상' : '공사규모_1000-'
                }

costlist = ['공사규모_0-5','공사규모_5-10','공사규모_10-30','공사규모_30-50','공사규모_50-100','공사규모_100-500','공사규모_500-1000','공사규모_1000-']            
for idx, row in data.iterrows():
    checkkeys = list(costchecklist.keys())
    
    rangename = ""
    if row['공사비'] in checkkeys:
        rangename = costchecklist[row['공사비']]
    else : 
        rangename = '공사규모_0-5'

    for name in costlist:
        if name == rangename:
            data.at[idx, name] = 1
        else:
            data.at[idx, name] = 0

data[costlist]
# type(data.at[0,'공사규모_5-10'])

,공사규모_0-5,공사규모_5-10,공사규모_10-30,공사규모_30-50,공사규모_50-100,공사규모_100-500,공사규모_500-1000,공사규모_1000-
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
14284,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
14285,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
14286,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
14287,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
dateformat = "%Y-%m-%d"


for idx, row in data.iterrows():
    if isNaN(row['해당공종 공사시작일']) or isNaN(row['해당공종 공사종료일']) :
        data.at[idx, '공사기간'] = 0
    else:
        start = datetime.strptime(row['해당공종 공사시작일'], dateformat)
        end = datetime.strptime(row['해당공종 공사종료일'], dateformat)
        daydiff = end - start       
        data.at[idx, '공사기간'] = daydiff.days


# print(data['Month'].value_counts())
data['공사기간'].value_counts()

0.0       505
30.0      195
60.0      169
364.0     140
29.0      132
         ... 
1013.0      1
1455.0      1
1862.0      1
1398.0      1
1108.0      1
Name: 공사기간, Length: 1448, dtype: int64

In [45]:
# droplist = []
# for idx, row in data.iterrows():
#     if row['온도'] == '미입력': 
#         droplist.append(idx)
# # print(droplist)
# data = data.drop(droplist, axis = 0)

# data['온도'] = data['온도'].astype(float)

# for idx, row in data.iterrows():
#     if (row['온도'] > 50):
#         data.at[idx, '온도'] = 50.0
#     if (row['온도'] < -15):
#         data.at[idx, '온도'] = -20.0
        

# # data
# data['온도'].value_counts()

 20.0    820
 25.0    760
 15.0    640
 10.0    615
 28.0    563
        ... 
-0.9       1
-5.2       1
-5.8       1
-6.4       1
-14.0      1
Name: 온도, Length: 338, dtype: int64

In [27]:
data.to_csv('Results/20221118_before_drop.csv')

In [59]:
data = pd.read_csv("Results/20221118_before_drop.csv")
data = data.drop(['해당공종 공사시작일','해당공종 공사종료일','공사비','공종(대분류)','공종(소분류)','10이상 20미만 사망자', '10이상 20미만 부상자','시설물 중분류', '시설물 소분류'], axis=1)

In [60]:

data = data.drop(['작업자수',
 '안전관리계획',
 '사고조사방법',
 '향후조치계획',
 '사고경위',
 '사고발생후 조치사항',
 '재발방지대책','피해금액',
 '피해내용',
 '사고신고사유',
 '해당공종 공사비',
 '공사시작일',
 '공사종료일',
 '낙찰율','내국인 부상자',
 '외국인 부상자',
 '남성 부상자',
 '여성 부상자','내국인 사망자','외국인 사망자','남성 사망자','여성 사망자','주원인 유형','사고원인(주원인)','사고원인(보조원인1)','사고원인(보조원인2)','구체적사고원인',
 '사고위치 장소','사고위치 장소(직접입력)','사고위치 부위','사고위치 부위(직접입력)','사고명','사고일시','습도','연면적','지상층수','지하층수','공사종류','안전방호조치여부','개인보호조치여부','인적사고종류','물적사고종류','사고객체(대분류)','사고객체(소분류)'],axis=1)
data

,Unnamed: 0,공공민간구분,날씨,온도,시설물 분류,작업프로세스,사망자,20이상 30미만 사망자,30이상 40미만 사망자,40이상 50미만 사망자,...,공종,공사규모_0-5,공사규모_5-10,공사규모_10-30,공사규모_30-50,공사규모_50-100,공사규모_100-500,공사규모_500-1000,공사규모_1000-,공사기간
0,0,민간,맑음,23.0,기타 건축,해체작업,0,0,0,0,...,기타공사,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,744.0
1,1,민간,강우,28.0,공동주택,해체작업,0,0,0,0,...,가설공사,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,2,민간,맑음,30.0,토목,해체작업,0,0,0,0,...,가설공사,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,244.0
3,3,공공,맑음,23.0,토목,해체작업,0,0,0,0,...,가설공사,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1565.0
4,4,민간,맑음,26.0,공동주택,설비작업,0,0,0,0,...,설비공사,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,882.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14284,14284,민간,맑음,20.0,기타 건축,마감작업,0,0,0,0,...,미장공사,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,44.0
14285,14285,공공,맑음,20.0,기타 건축,해체작업,0,0,0,0,...,콘크리트공사,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,182.0
14286,14286,공공,흐림,15.0,토목,조립작업,0,0,0,0,...,콘크리트공사,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,70.0
14287,14287,민간,맑음,12.0,기타 건축,설치작업,0,0,0,0,...,타일 및 테라코타공사,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0


In [11]:
data.to_csv('Results/20221118.csv')

In [10]:
data = pd.read_csv("Results/20221118.csv", index_col = 0)
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
data

,공공민간구분,날씨,온도,시설물 분류,작업프로세스,사망자,20이상 30미만 사망자,30이상 40미만 사망자,40이상 50미만 사망자,50이상 60미만 사망자,...,공종,공사규모_0-5,공사규모_5-10,공사규모_10-30,공사규모_30-50,공사규모_50-100,공사규모_100-500,공사규모_500-1000,공사규모_1000-,공사기간
0,민간,맑음,23.0,기타 건축,해체작업,0,0,0,0,0,...,기타공사,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,744.0
1,민간,강우,28.0,공동주택,해체작업,0,0,0,0,0,...,가설공사,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,민간,맑음,30.0,토목,해체작업,0,0,0,0,0,...,가설공사,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,244.0
3,공공,맑음,23.0,토목,해체작업,0,0,0,0,0,...,가설공사,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1565.0
4,민간,맑음,26.0,공동주택,설비작업,0,0,0,0,0,...,설비공사,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,882.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14284,민간,맑음,20.0,기타 건축,마감작업,0,0,0,0,0,...,미장공사,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,44.0
14285,공공,맑음,20.0,기타 건축,해체작업,0,0,0,0,0,...,콘크리트공사,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,182.0
14286,공공,흐림,15.0,토목,조립작업,0,0,0,0,0,...,콘크리트공사,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,70.0
14287,민간,맑음,12.0,기타 건축,설치작업,0,0,0,0,0,...,타일 및 테라코타공사,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0


In [17]:
data['공종(소분류)'].value_counts()

철근콘크리트공사        3948
기타              1651
가설공사            1586
해체 및 철거공사        695
기계설비공사           539
토공사              530
철골공사             439
건축 토공사           367
수장공사             363
관공사              360
전기설비공사           345
미장공사             315
건축물 부대공사         312
타일 및 돌공사         280
도로 및 포장공사        242
교량공사             219
산업설비공사           202
도장공사             196
터널공사             177
목공사              173
조적공사             172
창호 및 유리공사        142
하천공사             135
금속공사             129
관공사 부대공사         121
항만공사             112
방수공사             110
지붕 및 홈통공사         99
말뚝공사              66
지정공사              51
철도 및 궤도공사         49
조경공사              45
지반개량공사            28
지반조사              27
강구조물공사            26
프리캐스트 콘크리트공사      11
댐 및 제방공사          10
미입력                7
통신설비공사             6
특수 건축물공사           4
Name: 공종(소분류), dtype: int64

In [67]:
data[['물적사고종류','인적사고종류']].loc[data['물적사고종류'] == "기타"]

,물적사고종류,인적사고종류
0,기타,끼임
1,기타,물체에 맞음
2,기타,떨어짐
3,기타,"절단, 베임"
4,기타,넘어짐
...,...,...
14284,기타,떨어짐
14285,기타,부딪힘
14286,기타,끼임
14287,기타,떨어짐


In [26]:
dateformat = "%Y%m%d"
date = datetime.strptime('20191130', dateformat)
date = date + timedelta(days=1)
date.strftime(dateformat)


'20191201'